In [1]:
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Conv2D, concatenate, GlobalAveragePooling2D, Dense, MaxPooling2D, Dropout, Flatten, ReLU, Add, BatchNormalization, AveragePooling2D, Multiply, GlobalAveragePooling2D, Reshape,DepthwiseConv2D
from keras.optimizers import RMSprop
import keras.backend as K
import cv2
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from PIL import Image
from tensorflow.keras.metrics import Metric,Precision, Recall
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3, MobileNet
import pandas as pd

2023-12-13 16:43:17.738478: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-13 16:43:18.455646: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
train_dir = r'/home/featurize/data/Training'
test_dir = r'/home/featurize/data/Test'
#The total number of images: 90483.
#Training set size: 67692 images (one fruit or vegetable per image).
#Test set size: 22688 images (one fruit or vegetable per image).
#The number of classes: 131 (fruits and vegetables).
#Image size: 100x100 pixels.

In [3]:
#find out the total calsses of dataset.
className = glob.glob(train_dir + "/*")
numberOfClass = len(className)
print("NumberOfClass: ",numberOfClass)

NumberOfClass:  131


In [4]:
def denoise_background(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    _, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)
    thresh_rgb = cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)
    np.copyto(image, thresh_rgb, where=thresh_rgb == 255)
    return image

In [5]:
# Load and preprocess the training data
train_datagen = ImageDataGenerator(
    preprocessing_function=denoise_background,
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    validation_split=0.068,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(100, 100),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb', 
    subset='training' 
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(100,100),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb', 
    subset='validation'
)

# Load and preprocess the test data
test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(100,100),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb'
)

Found 63145 images belonging to 131 classes.
Found 4547 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


In [6]:
input_shape = Input(shape=(100,100,3))

In [7]:
class CustomAccuracy(Metric):
    def __init__(self, name='accuracy', **kwargs):
        super(CustomAccuracy, self).__init__(name=name, **kwargs)
        self.correct = self.add_weight(name='correct', initializer='zeros')
        self.total = self.add_weight(name='total', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_classes = tf.argmax(y_pred, axis=1)
        y_true_classes = tf.argmax(y_true, axis=1)
        correct_predictions = tf.equal(y_pred_classes, y_true_classes)
        correct_predictions = tf.cast(correct_predictions, 'float32')
        
        self.correct.assign_add(tf.reduce_sum(correct_predictions))
        self.total.assign_add(tf.cast(tf.size(y_true_classes), 'float32'))

    def result(self):
        return tf.divide(self.correct, self.total)

    def reset_state(self):
        self.correct.assign(0)
        self.total.assign(0)


In [8]:
class CustomCrossEntropyLoss(tf.keras.losses.Loss):
    def __init__(self, name='loss', **kwargs):
        super(CustomCrossEntropyLoss, self).__init__(name=name, **kwargs)

    def call(self, y_true, y_pred):
        epsilon = 1e-10
        y_pred = tf.clip_by_value(y_pred, epsilon, 1 - epsilon)

        loss = -tf.reduce_sum(y_true * tf.math.log(y_pred), axis=-1)

        return loss

Add the output layer for the other models

In [9]:
def add_new_last_layer(base_model):
    x = Dense(128, activation='relu')(base_model.output) 
    x = BatchNormalization()(x) 
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x) 
    x = Dropout(0.3)(x)
    last_output= Dense(numberOfClass, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=last_output)
    return model 

def prepare_model(model):
    model = add_new_last_layer(model)
    model.compile(optimizer=RMSprop(learning_rate=0.001), loss=CustomCrossEntropyLoss(),metrics=[CustomAccuracy(), Precision(name='precision'),Recall(name='recall')])
    return model

In [10]:
mobilenet_model = prepare_model(MobileNet(weights=None, include_top=False, input_shape=(100,100,3)))

2023-12-13 16:43:37.086413: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-13 16:43:37.122510: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-13 16:43:37.122755: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Train the mobienet model

In [11]:
history_mobilenet=mobilenet_model.fit(train_generator, epochs=40, validation_data=validation_generator)

Epoch 1/40


2023-12-13 16:43:43.913156: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-12-13 16:43:45.964785: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-12-13 16:43:46.241315: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1cac1b10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-13 16:43:46.241386: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2023-12-13 16:43:46.277455: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-13 16:43:46.493883: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the proce

1974/1974 [==============================] - 199s 94ms/step - loss: 1.5375 - accuracy: 0.5521 - precision: 0.7531 - recall: 0.4526 - val_loss: 1.2243 - val_accuracy: 0.6774 - val_precision: 0.7047 - val_recall: 0.6576
Epoch 2/40
1974/1974 [==============================] - 171s 87ms/step - loss: 0.4380 - accuracy: 0.8558 - precision: 0.8780 - recall: 0.8365 - val_loss: 0.5387 - val_accuracy: 0.8379 - val_precision: 0.8530 - val_recall: 0.8285
Epoch 3/40
1974/1974 [==============================] - 176s 89ms/step - loss: 0.2603 - accuracy: 0.9153 - precision: 0.9247 - recall: 0.9073 - val_loss: 0.2903 - val_accuracy: 0.9153 - val_precision: 0.9194 - val_recall: 0.9101
Epoch 4/40
1974/1974 [==============================] - 169s 86ms/step - loss: 0.1898 - accuracy: 0.9393 - precision: 0.9448 - recall: 0.9348 - val_loss: 0.2994 - val_accuracy: 0.9219 - val_precision: 0.9253 - val_recall: 0.9202
Epoch 5/40
1974/1974 [==============================] - 156s 79ms/step - loss: 0.1485 - accurac

In [12]:
#save the history of mobilenet    
save_history_mobilenet = pd.DataFrame(history_mobilenet.history)

save_history_mobilenet.to_csv("mobilenet_model_training_history_40epoch_1211.csv", index=False)

save_path=r'mobilenet_model1213_40epochs.h5'
mobilenet_model.save(save_path)

/environment/miniconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
